# ommx.v1.InstanceをQUBOに変換する

## ペナルティ法

今回扱うOpenJijを含めて、多くのサンプラーと呼ばれるソフトウェアは制約条件を含まない定式化である QUBO (Quadratic Unconstrained Binary Optimization) で記述された目的関数を小さくするようなサンプルを生成する形で動作します。上で定式化した巡回セールスマン問題は変数は全てバイナリですが、制約条件があるためUnconstrainedではありません。そこでペナルティ法によって制約条件を目的関数に押し込む事で制約なし問題に変換します。OMMXの [`Instance.uniform_penalty_method`](https://jij-inc.github.io/ommx/python/ommx/autoapi/ommx/v1/index.html#ommx.v1.Instance.uniform_penalty_method) は等式制約付き問題

$$
\begin{align*}
\min \quad &f(x) \\
\text{s.t.} \quad &g_i(x) = 0 \quad (\forall i)
\end{align*}
$$

を一つのパラメータ $\lambda$ を持つ制約なし問題

$$
\min \quad f(x) + \lambda \sum_i g_i(x)^2
$$

に変換します。また重みパラメータを制約毎に指定したい場合は [`Instance.penalty_method`](https://jij-inc.github.io/ommx/python/ommx/autoapi/ommx/v1/index.html#ommx.v1.Instance.penalty_method) を使うと

$$
\min \quad f(x) + \sum_i \lambda_i g_i(x)^2
$$

という形に変換できます。

これは決定変数でないパラメータをもっているため `ommx.v1.Instance` ではなくて `ommx.v1.ParametricInstance` という型になっていて、次のパラメータ付きQUBOに対応します。

$$
\min \quad \sum_{t=0}^{N-1} \sum_{i, j = 0}^{N-1} d(i, j) x_{t, i} x_{(t+1 \% N), j}
+ \lambda \left[ \sum_{t=0}^{N-1} \left(\sum_{i=0}^{N-1} x_{t, i} - 1\right)^2
+ \sum_{i=0}^{N-1} \left(\sum_{t=0}^{N-1} x_{t, i} - 1\right)^2 \right]
$$

この `ParametricInstance` がどのようなパラメータを持っているかは `parameters` プロパティで確認できます。

ただしこの `qubo` インスタンスは元の制約条件の情報を `removed_constraints` として保持しています。これはQUBOに対してサンプリングした結果が元の問題の制約条件を満たしているのかどうかを検証するための情報です。QUBOへの変換は極端な例ですが、このように数理モデルに前処理を加えることでユーザーが入力した制約条件がソルバーに渡す前に不必要になってしまうことはよくあります。このような場合でもユーザーは自分が入力した制約条件について興味があるので `ommx.v1.Instance` ではこれらの情報を保持する仕組みが備わっています。